## N = 15

In [1]:
import numpy as np

# load date
X = np.load("Datasets/kryptonite-15-X.npy")
X.shape

(30000, 15)

In [2]:
y = np.load("Datasets/kryptonite-15-y.npy")
y.shape

(30000,)

In [3]:
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader, random_split

# Convert numpy arrays to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float)
y_tensor = torch.tensor(y, dtype=torch.float)

# create a TensorDataset
dataset = TensorDataset(X_tensor, y_tensor)

# define split sizes (70% train, 15% validation, 15% test)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

# Split the dataset into train, validation, and test
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoaders for each subset
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# check data loader output
for X_batch, y_batch in tqdm(train_loader):
    print(X_batch.shape, y_batch.shape)  # X_batch is of shape [batch_size, *input_shape]
    break


  0%|          | 0/165 [00:00<?, ?it/s]

torch.Size([128, 15]) torch.Size([128])


In [4]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# build model for training
class MLP(nn.Module):
    def __init__(self, input_size=15, hidden_size=128, output_size=1):
        super(MLP, self).__init__()

        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, hidden_size*2)
        self.layer3 = nn.Linear(hidden_size*2, hidden_size*2)
        self.layer4 = nn.Linear(hidden_size*2, hidden_size)
        self.layer5 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        x = F.relu(self.layer3(x))
        x = F.relu(self.layer4(x))
        x = torch.sigmoid(self.layer5(x))  # sigmoid activation for binary output
        return x


In [5]:
# initialize the model, loss function, and optimizer
model = MLP(input_size=15, hidden_size=128, output_size=1)
loss_fn = nn.BCELoss()  # binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1)

In [6]:
torch.manual_seed(42)

# set the number of epochs
epochs = 100
best_acc = 0

for epoch in range(epochs):
    """ Training """
    model.train()

    # forward pass
    correct = 0
    total = 0
    train_loss = 0
    for X_batch, y_batch in train_loader:
        output = model(X_batch).squeeze()
        y_preds = torch.round(output)

        correct += torch.eq(y_preds, y_batch).sum().item()
        total += len(y_batch)

        loss = loss_fn(output, y_batch)
        train_loss += loss.item()
        # zero the optimizer
        optimizer.zero_grad()
        # backpropagattion
        loss.backward()
        # GD
        optimizer.step()

    scheduler.step()
    train_acc = (correct / total) * 100

    """ Testing """
    model.eval()
    correct = 0
    total = 0
    test_loss = 0
    with torch.inference_mode():
        for X_batch, y_batch in test_loader:
            output = model(X_batch).squeeze()
            y_preds = torch.round(output)

            correct += torch.eq(y_preds, y_batch).sum().item()
            total += len(y_batch)

            loss = loss_fn(output, y_batch)
            test_loss += loss.item()
        
        test_acc = (correct / total) * 100
        if test_acc > best_acc:
            best_acc = test_acc
            torch.save(model.state_dict(), "n-15best.pth")

    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Train Loss: {train_loss:.5f} | Acc: {train_acc:.2f}% | Learning Rate: {scheduler.get_last_lr()[0]:.7f} | Test loss: {test_loss:.5f} | Test Acc: {test_acc:.2f}%")

Epoch: 0 | Train Loss: 114.38105 | Acc: 50.27% | Learning Rate: 0.0001000 | Test loss: 24.95025 | Test Acc: 50.96%
Epoch: 10 | Train Loss: 114.16007 | Acc: 52.02% | Learning Rate: 0.0001000 | Test loss: 24.95427 | Test Acc: 49.96%
Epoch: 20 | Train Loss: 52.22822 | Acc: 88.21% | Learning Rate: 0.0001000 | Test loss: 12.18556 | Test Acc: 87.47%
Epoch: 30 | Train Loss: 36.82878 | Acc: 92.94% | Learning Rate: 0.0001000 | Test loss: 10.76105 | Test Acc: 89.47%
Epoch: 40 | Train Loss: 28.37321 | Acc: 95.05% | Learning Rate: 0.0000100 | Test loss: 9.41182 | Test Acc: 91.84%
Epoch: 50 | Train Loss: 27.38853 | Acc: 95.30% | Learning Rate: 0.0000100 | Test loss: 9.53573 | Test Acc: 91.98%
Epoch: 60 | Train Loss: 26.50051 | Acc: 95.56% | Learning Rate: 0.0000100 | Test loss: 9.59674 | Test Acc: 91.98%
Epoch: 70 | Train Loss: 25.96151 | Acc: 95.62% | Learning Rate: 0.0000100 | Test loss: 9.70565 | Test Acc: 91.91%
Epoch: 80 | Train Loss: 25.43925 | Acc: 95.78% | Learning Rate: 0.0000010 | Test lo

In [7]:
model = MLP(input_size=15, hidden_size=128, output_size=1)
model.load_state_dict(torch.load("n-15best.pth"))
model.eval()

correct = 0
total = 0
val_loss = 0
with torch.inference_mode():
    for X_batch, y_batch in tqdm(val_loader):
        output = model(X_batch).squeeze()
        y_preds = torch.round(output)

        correct += torch.eq(y_preds, y_batch).sum().item()
        total += len(y_batch)

        loss = loss_fn(output, y_batch)
        val_loss += loss.item()
    
    val_acc = (correct / total) * 100

print(f"Validation Loss: {val_loss:.5f} | Validation Acc: {val_acc:.2f}%")

100%|██████████| 36/36 [00:00<00:00, 1615.37it/s]

Validation Loss: 8.57605 | Validation Acc: 92.56%
